In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

C:\Users\richa\Documents\GitHub\transformers-interpret


In [16]:
from bigbio.dataloader import BigBioConfigHelpers
conhelps = BigBioConfigHelpers()
dataset_name = 'cadec_bigbio_kb'
dataset = conhelps.for_config_name(dataset_name).load_dataset()

Reusing dataset cadec (C:\Users\richa\.cache\huggingface\datasets\cadec\cadec_bigbio_kb\1.0.0\aa0c4972a66d4e69245748c72bda714effb7d8c98aea405f186dd03cb488dbb0)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
len(dataset)

1

In [4]:
# from datasets import load_dataset

# dataset = load_dataset("yelp_review_full")

# dataset["train"][100]

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline, AutoConfig
huggingface_model = 'dslim/bert-base-NER'
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(huggingface_model)

In [6]:
from transformers_interpret.evaluation.input_pre_processor import InputPreProcessor, get_labels_from_dataset
label2id, id2label = get_labels_from_dataset(dataset)
print(label2id)
pre_processor = InputPreProcessor(tokenizer, label2id)

{'O': 0, 'ADR': 1, 'Disease': 2, 'Drug': 3, 'Finding': 4, 'Symptom': 5}


In [7]:
model: AutoModelForTokenClassification = AutoModelForTokenClassification.from_pretrained(huggingface_model, ignore_mismatched_sizes=True, num_labels=len(label2id))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# for split in dataset:
#     empty_documents = []
#     print(len(dataset[split]))
#     for i, doc in enumerate(dataset[split]):
#         doc = doc | pre_processor(doc)
#         print(doc)
#         break
#         if len(doc['text']) == 0:
#             empty_documents.append(i)
#     dataset[split] = dataset[split].select([i for i in range(len(dataset[split])) if i not in empty_documents])
#     print(len(dataset[split]))
#     dataset[split].feature = None

In [9]:
def tokenize_function(examples):
    return pre_processor(examples)
#     return tokenizer(passages, padding="max_length", truncation=True)  # ["passages"][0][0]

dataset_length = len(dataset["train"])
tokenized_datasets = dataset.map(tokenize_function)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(dataset_length//2))
eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(dataset_length//2, dataset_length))
print(train_dataset)

Parameter 'function'=<function tokenize_function at 0x00000215AA6CB4C0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1250 [00:00<?, ?ex/s]

Loading cached shuffled indices for dataset at C:\Users\richa\.cache\huggingface\datasets\cadec\cadec_bigbio_kb\1.0.0\aa0c4972a66d4e69245748c72bda714effb7d8c98aea405f186dd03cb488dbb0\cache-04b4a739e075869a.arrow


Dataset({
    features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'],
    num_rows: 625
})


In [10]:
print(train_dataset[10])

{'id': '593', 'document_id': 'LIPITOR.48', 'passages': [{'id': 'LIPITOR.48__text', 'type': 'abstract', 'text': ['Stopped because of HIGH CPK.\nMy Liver blood test are also mildly elevated.\nI was in deniel that the statins would cause ME side effects- I have Terrible muscle and joint pain, Burning sensations in neck shoulders and upper chest.\nPalpatations .\nInsomnia because can not sleep from pain.\nI do not want to take pain killers all the time since they have side effects too.\nWeight gain -probably not associated.\nIOne of the reasons I left my job is because I was so weak all the time and had difficulty walking and driving.\nI was a Sales rep for a drug company and many years ago I promoted Zocor.\nHow IRONIC that I should suffer from statins my self.\nMy life is miserable.\nI feel like dying.\nI am sorry I did not keep my weight low so as to avoid my lipids to go so high.\nThat way I would have never had to suffer like I am now.\nThe doctors just say and my husband OH your in p

In [11]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [12]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    print(eval_pred, eval_pred.shape)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: events, entities, coreferences, document_id, offset_mapping, text, id, relations, passages. If events, entities, coreferences, document_id, offset_mapping, text, id, relations, passages are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
C:\Users\richa\Documents\GitHub\transformers-interpret\.env\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 625
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimizat

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [20]:
import os, datetime
os.makedirs(f"trained_models", exist_ok=True)
model.save_pretrained(f"trained_models/{dataset_name}_{str(datetime.datetime.now()).replace(':', '_')}")

Configuration saved in trained_models/cadec_bigbio_kb_2022-07-08 17_50_33.139013\config.json
Model weights saved in trained_models/cadec_bigbio_kb_2022-07-08 17_50_33.139013\pytorch_model.bin
